## Primality-testing and modular arithmetic

This Jupyter notebook provides a toolbox for exploring the Fermat-Euler theorem, primality-testing, and more.

Python is very convenient for modular arithmetic.  Try the following commands.

To run a command, click in the cell to make sure it's active.  Then press shift-enter to run the command(s) in the cell.  Discuss the results with others in the class.

In [ ]:
23 % 5  # What do you think the % operator does?

In [ ]:
divmod(23,5) # Sometimes you care about quotient and remainder...

In [ ]:
pow(2,5) # Python-power!  What does it do?

In [ ]:
pow(2,90) % 91 # What do you get?

The output should be "64L".  The letter L stands for "Long".  It's there because Python had to switch into long-integer mode along the way, to deal with the large number pow(2,90).  

Much faster, for people and computers, is to keep the computations in the modular world throughout.  So try the following.

In [ ]:
pow(2,90,91)

Notice that "L" is no longer there.  Along the way, Python kept the computations "mod 91", so it didn't have to work with numbers bigger than 91.  No long integers needed!

Pow commands are very fast, in modular arithmetic.  Try something like this, if you dare.  Just don't forget all three inputs to the pow command.

In [ ]:
pow(2347883845,23428923749827398472938742,2394872934872983729837423429348472)

Pow can be used for primality testing.  Compute the following.  What do these computations tell you about 31, 51, 91?  

In [ ]:
pow(3,30,31)

In [ ]:
pow(7,50,51)

In [ ]:
pow(3,90,91)

We say that a number b witnesses the non-primality of a number n, if pow(b,n-1,n) does not equal 1.  Indeed, this means that n cannot be prime, by Fermat's Little Theorem.  We can turn this into a Python function with the code below.  Make sure to copy it carefully, using all indentation as below.

Press shift-enter as usual to run the code.  Nothing will happen on the screen, but you have just taught the computer a new trick, called "witness".  

In [ ]:
def witness(b,n):
    # A function which outputs False if n cannot be prime.
    if pow(b,n-1,n) == 1:
        return True
    else:
        return False

Now we can use our function, instead of pow commands.

In [ ]:
witness(2,91)

In [ ]:
witness(3,91)

So 2 witnesses the non-primality of 91, but 3 does not.  We might say that 91 "looks prime" to the witness 3.  

Some numbers -- the Carmichael numbers -- look prime to a lot of witnesses.  An example is 41041.

In [ ]:
witness(2,41041)

In [ ]:
witness(3,41041)

In [ ]:
witness(4,41041)

In [ ]:
witness(5,41041)

In [ ]:
witness(6,41041)

To improve this primality test, we make our witnesses more perceptive using an idea of Miller and Rabin.  To use the Miller-Rabin test with base b and test-number n, compute pow(b,k,n) as k ranges over the set (n-1), (n-1)/2, (n-1)/4, etc., as long as those numbers are integers.

In [ ]:
pow(2,41040,41041)

In [ ]:
pow(2,20520, 41041)

In [ ]:
pow(2,10260,41041)

In [ ]:
pow(2,5130,41041)

Well, that's odd!  If x = pow(2,5130,41041), then the square of x equals pow(2,10260,41041).  So 27182 squared equals 1.  This violates the ROO property of primes:

(ROO)  If p is prime, and x^2 is congruent to 1 mod p, then x is congruent to 1 or -1 mod p.

Hence 41041 is not prime!

If you don't trust this, let's do a *brute force* check.  Can you see how this works?

In [ ]:
from math import sqrt

In [ ]:
for j in range(2,int(sqrt(41041))):
    if 41041%j == 0:
        print j,"is a divisor of 41041."

So we can find factors of 41041.
Let's prove that 561 is not prime, using the ROO property.

In [ ]:
pow(2,560,561) # 561 passes the basic witness test, using 2 as a base.

In [ ]:
pow(2,280,561) # Divide the exponent by two at each step.

In [ ]:
pow(2,140,561) # Uh oh.  prime-fail!  What does this tell you about 561?  Why?

The following function will carry this process out automatically.  Using strong_witness(b,n) will output False if the base b detects the nonprimality of n.

In [ ]:
def strong_witness(b,n):
    looks_prime = True # n looks prime until it's proven not prime.

    # Step one:  factoring out a power of 2 from n-1.
    e = 0
    m = n-1  
    while m%2 == 0:  # As long as m is even. 
        e += 1
        m = m//2 # Integer division, to be safe in Python 2.7 or 3.x.
    # The result of the above process is that
    # n-1 = 2^e * m, and m is odd.
    
    # Step two: computing b^m mod n
    s = pow(b,m,n)

    # Step three:  successive squaring, to look for ROO violations.
    k = 0
    while (k < e) and looks_prime:
        ss = (s*s)%n
        if (ss == 1) and (s != 1) and (s != n-1):
            looks_prime = False
            # Note:  add parentheses to all print commands, if you're using Python 3.x instead of 2.7.
            # See [https://docs.python.org/3/whatsnew/3.0.html] for more details.
            print b,"^(",m*pow(2,k),") = ",s," and ",s,"^2 = ",ss," modulo ",n
            print "This violates ROO."
        s = ss
        k += 1
    
    # Step four:  if no ROO violations, check for FLT violation.
    if looks_prime:
        if s != 1:
            print b,"^",m*pow(2,k)," is not congruent to 1, modulo ",n
            print "This violates Fermat's Little Theorem"
            looks_prime = False
    
    if looks_prime:
        print n," might be prime."
    else:
        print n," is definitely not prime."
    return looks_prime
             
        

In [ ]:
strong_witness(2,561)

In [ ]:
strong_witness(3,340561)

Let's try the Miller-Rabin test on some large random numbers.  First, import the random package.

In [ ]:
from random import *

In [ ]:
p = randint(pow(10,100), pow(10,101)) # randint(x,y) chooses a random integer between x and y.

In [ ]:
strong_witness(2,p) # Is this randomly chosen integer prime?

The following function will find a big prime.  Try it with 10 digits or 100 digits.  DO NOT try it with more than 10000 digits or so, or else your computer might hang.

Before running it, you may wish to delete all the "print" commands from the strong_witness function, above (and shift-enter that cell).  

In [ ]:
def find_big_prime(digits, maxtries=100, witnesses=30): # finds a prime number with a given number of digits.
    prime_found = False
    tries = 0
    while (prime_found == False) and (tries < maxtries):
        tries += 1
        p = randint(10**digits, 10**(digits+1))
        is_prime = True
        nw = 0 # how many witnesses have we tried?
        while is_prime and (nw < witnesses): # We won't try more than 30 witnesses!
            nw += 1
            w = randint(2,p-1) # Choose a random witness.
            is_prime = strong_witness(w,p) # Run the Miller-Rabin test.
        if is_prime: # If it passed 20 iterations of Miller-Rabin...
            prime_found = True
            print p,"is prime, with chance of error",(pow(0.25,witnesses)),"!"
            return p
    print "I tried and tried but didn't find a prime."
    return None


In [ ]:
find_big_prime(10)